In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用人脸风格化器的顶点 AI 模型花园 MediaPipe

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_mediapipe_face_stylizer.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>

  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_mediapipe_face_stylizer.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_mediapipe_face_stylizer.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
在 Vertex AI Workbench 中打开
    </a>
  </td>
</table>

**_注意_**：此笔记本已在以下环境中进行测试：

* Python 版本 = 3.9

**_注意_**：在该 Colab 中链接的检查点和数据集不是由谷歌拥有或分发的，是由第三方提供的。在使用检查点和数据之前，请先查看第三方提供的条款和条件。

## 概述

本手册演示如何使用[MediaPipe模型制作器](https://developers.google.com/mediapipe/solutions/model_maker)在Vertex AI模型花园中自定义一个设备上的面部风格化器模型。

MediaPipe面部风格化解决方案提供了几个可以立即使用的模型，您可以在您的应用程序中将脸部转换为包括卡通、油画等风格。但是，如果您需要将面部转换为提供的模型未涵盖的未见风格，您可以使用自己的数据和MediaPipe模型制作器自定义预训练模型。这个模型修改工具会使用您提供的数据微调模型的一部分。这种方法比从头开始训练一个新模型更快，可以生成一个适用于您特定应用程序的模型。

以下部分将向您展示如何使用模型制作器在Vertex AI上使用您自己的数据重新训练预构建的面部风格化模型，然后您可以将其用于MediaPipe面部风格化器。

### 目标

* 自定义一个面部风格化器模型
  * 将输入数据转换为训练格式
  * 创建[自定义工作](https://cloud.google.com/vertex-ai/docs/training/create-custom-job)以自定义新模型
  * 导出定制模型

* 清理资源

### 成本

本教程使用谷歌云的计费组件：

* Vertex AI
* 云存储

了解[Vertex AI价格](https://cloud.google.com/vertex-ai/pricing)和[云存储价格](https://cloud.google.com/storage/pricing)，并使用[定价计算器](https://cloud.google.com/products/calculator/)根据您的预计使用量生成成本估算。

在你开始之前

只能在Colab上运行
运行以下命令安装依赖项并与Google Cloud进行身份验证。

In [ ]:
! pip3 install --upgrade pip

import sys

if "google.colab" in sys.modules:
    ! pip3 install --upgrade google-cloud-aiplatform

    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

    from google.colab import auth as google_auth

    google_auth.authenticate_user()

#### 设置您的项目ID

**如果您不知道您的项目ID**，请参考支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

区域

您还可以更改 Vertex AI 使用的“REGION” 变量。了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}
REGION_PREFIX = REGION.split("-")[0]
assert REGION_PREFIX in (
    "us",
    "europe",
    "asia",
), f'{REGION} is not supported. It must be prefixed by "us", "asia", or "europe".'

创建一个云存储桶

创建一个存储桶，用于存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的存储桶不存在时才运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

### 导入库

In [ ]:
import json
import os
from datetime import datetime

from google.cloud import aiplatform

### 初始化 Python 的 Vertex AI SDK

为您的项目初始化 Python 的 Vertex AI SDK。

In [ ]:
now = datetime.now().strftime("%Y%m%d-%H%M%S")

STAGING_BUCKET = os.path.join(BUCKET_URI, "temp/%s" % now)


EXPORTED_MODEL_OUTPUT_DIRECTORY = os.path.join(STAGING_BUCKET, "model")
EXPORTED_MODEL_OUTPUT_FILE = os.path.join(
    EXPORTED_MODEL_OUTPUT_DIRECTORY, "model.tflite"
)

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

定义培训机器规格

In [ ]:
TRAINING_JOB_DISPLAY_NAME = "mediapipe_face_stylizer_%s" % now
TRAINING_CONTAINER = f"{REGION_PREFIX}-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/mediapipe-train"
TRAINING_MACHINE_TYPE = "n1-highmem-16"
TRAINING_ACCELERATOR_TYPE = "NVIDIA_TESLA_V100"
TRAINING_ACCELERATOR_COUNT = 2

训练您定制的模型

准备训练的输入数据

重新训练面部风格化模型需要用户提供一张单独的风格化面部图片。期望风格化面部是正面朝向，左右眼睛和嘴巴可见。面部旋转应该很小，即偏航、俯仰和横滚轴的旋转小于30度。

您可以将图片上传到Google云存储，或使用我们提供的示例图片链接。

In [ ]:
training_data_path = "gs://mediapipe-assets/face_stylizer_style_color_sketch.jpg"  # @param {type:"string"}

重新训练模型

一旦您提供了输入图片，您可以开始重新训练人脸风格模型以适应新的风格。这种模型修改称为迁移学习。以下说明使用上一节准备好的数据来重新训练一个人脸风格模型，以将漫画风格应用到原始人脸上。

**_注意_**：对于这种类型的模型，重新训练过程会使模型忘记它之前可以应用的任何风格。一旦重新训练完成，新模型只能应用由新风格图片定义的新风格。

## 设置重新训练选项
除了您的训练数据集之外，运行重新训练还需要一些必需的设置：

* **交换层次:** `swap_layers`参数用于确定如何在学习的风格和原始面部图像之间混合潜在代码层。潜在代码被表示为形状为[1, 12, 512]的张量。潜在代码张量的第二维称为层。面部风格化者通过在交换层上生成两个潜在代码的加权和来混合学习的风格和原始面部图像。因此，交换层是在[1, 12]之间的整数。设置更多层，将更多的风格应用于输出图像。尽管风格语义与层索引之间没有明确的映射，但是浅层，例如8、9，代表面部的全局特征，而深层，例如10、11，代表面部的细粒度特征。输出的风格化图像对交换层的设置敏感。默认情况下，设置为[8, 9, 10, 11]。
* **学习率和迭代次数:** 使用`learning_rate`和`epochs`来指定这两个超参数。learning_rate默认设置为4e-4。epochs定义了微调BlazeStyleGAN模型的迭代次数，默认设置为100。学习率越低，预期重新训练模型收敛的迭代次数就越大。
* **批大小:** `batch_size`用于定义我们围绕由编码器提取的潜在代码而采样的潜在代码样本的数量。潜在代码批用于微调解码器。通常批大小越大，性能就越好。它还受硬件内存的限制。对于A100 GPU，最大批大小为8。对于P100和T4 GPU，最大批大小为2。

您可以配置更进一步的高级参数，如`alpha`、`perception_loss_weight`、`adv_loss_weight`、`beta_1`和`beta_2`。

In [ ]:
# The layers of feature to be interpolated between encoding features and
# StyleGAN input features.
swap_layers: str = "[8, 9, 10, 11]"  # @param {type:"string"}
# The learning rate to use for gradient descent training.
learning_rate: float = 0.0001  # @param {type:"number"}
# Number of training iterations over the dataset.
epochs: int = 100  # @param {type:"slider", min:0, max:100, step:1}
# Batch size for training.
batch_size: int = 2  # @param {type:"number"}


# Other supported options

# Weighting coefficient of style latent for swapping layer interpolation.
# Its valid range is [0, 1]. The greater weight means stronger style is
# applied to the output image. Expect to set it to a small value,
# i.e. < 0.1.
alpha: float = 0.1  # @param {type:"number"}

# Weighting coefficients of image perception quality loss. It contains three
# coefficients, l1, content, and style which control the difference between the
# generated image and raw input image, the content difference between generated
# face and raw input face, and the how similar the style between the generated
# image and raw input image. Users can increase the style weight to enforce
# stronger style or the content weight to reserve more raw input face details.
# Weight for L1 loss.
perception_loss_l1: float = 0.5  # @param {type:"number"}
# Weight for content loss.
perception_loss_content: float = 4.0  # @param {type:"number"}
# Weight for stlye loss.
perception_loss_style: float = 1.0  # @param {type:"number"}

# Weighting coeffcieint of adversarial loss versus image perceptual quality loss.
# This hyperparameter is used to control the realism of the generated image. It
# expects a small value, i.e. < 0.2.
adv_loss_weight: float = 0.2  # @param {type:"number"}
# beta_1 used in tf.keras.optimizers.Adam.
beta_1: float = 0.0  # @param {type:"number"}
# beta_2 used in tf.keras.optimizers.Adam.
beta_2: float = 0.99  # @param {type:"number"}

### 重新训练
准备好您的训练数据集和重新训练选项后，您可以开始重新训练过程。该过程需要在 GPU 上运行，根据您的计算资源不同，可能需要几分钟到几个小时的时间。在 Vertex AI 上进行 GPU 处理，下面的示例重新训练大约需要 2 分钟。

要开始微调过程，请使用以下代码：

In [ ]:
model_export_path = EXPORTED_MODEL_OUTPUT_DIRECTORY

worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": TRAINING_MACHINE_TYPE,
            "accelerator_type": TRAINING_ACCELERATOR_TYPE,
            "accelerator_count": TRAINING_ACCELERATOR_COUNT,
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": TRAINING_CONTAINER,
            "command": [],
            "args": [
                "--task_name=face_stylizer",
                "--training_data_path=%s" % training_data_path,
                "--model_export_path=%s" % model_export_path,
                "--evaluation_result_path=%s" % model_export_path,
                "--hparams=%s"
                % json.dumps(
                    {
                        "learning_rate": learning_rate,
                        "batch_size": batch_size,
                        "epochs": epochs,
                        "beta_1": beta_1,
                        "beta_2": beta_2,
                    }
                ),
                "--model_options=%s"
                % json.dumps(
                    {
                        "swap_layers": json.loads(swap_layers),
                        "alpha": alpha,
                        "perception_loss_l1": perception_loss_l1,
                        "perception_loss_content": perception_loss_content,
                        "perception_loss_style": perception_loss_style,
                        "adv_loss_weight": adv_loss_weight,
                    }
                ),
            ],
        },
    }
]

training_job = aiplatform.CustomJob(
    display_name=TRAINING_JOB_DISPLAY_NAME,
    project=PROJECT_ID,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=STAGING_BUCKET,
)

training_job.run()

## 导出模型
重新训练模型后，您可以保存Tensorflow Lite模型，并按照[面部风格化任务指南](https://developers.google.com/mediapipe/solutions/vision/face_stylizer)将其集成到您的设备应用程序中。

In [ ]:
import sys

def copy_model(model_source, model_dest):
    ! gsutil cp {model_source} {model_dest}

copy_model(EXPORTED_MODEL_OUTPUT_FILE, "face_stylizer.task")

if "google.colab" in sys.modules:
    from google.colab import files

    files.download("face_stylizer.task")

清理

In [ ]:
# Delete training data and jobs.
if training_job.list(filter=f'display_name="{TRAINING_JOB_DISPLAY_NAME}"'):
    training_job.delete()

!gsutil rm -r {STAGING_BUCKET}